In [1]:
import os
import sys
import datetime
from pathlib import Path
import xml.etree.ElementTree as ET
import logging
import uuid
import time
import shapely
import pandas as pd
import geopandas as gpd
import seaborn
import dask
import numpy as np
import dask.dataframe as dd
from dask.distributed import Client

sys.path.insert(0, '..')
import src.hotspot_utils as util 
import src.process_nearest_hotspots as nearest_process
import src.xml_util as xutil

/env/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
outdir = Path(f"/home/jovyan/s3vt_dask/s3vtdata/workdir_test1")

In [3]:
logging.basicConfig(
    format='%(asctime)s [%(levelname)s] %(name)s - %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S',
)
_LOG = logging.getLogger(__name__)

In [4]:
# include n_workers equal or less than the number of core
client = Client(n_workers=8)
client

Client Scheduler: tcp://127.0.0.1:40559 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 16 Memory: 66.57 GB


# Processing Parameter used in Sub-setting Spatial Extent and Temporal Range for Area of Interest
##### The FRP data from nasa, esa, eumetsat and landgate are merged, sub-setted and neareast hotspots csv files are generated based on the parameters in `processing_parameters`  
##### The parameter `chunks` in blocking FRP data to enable multi-processing. If you encounter memory issues then higher the number.
##### The `start_time` and `end_time` can be used to subset for solar_day (3:00-22:00), solar_night (22:00-03:00 with 12 hours offset) and solar_all(0:00-24:00) hours.

In [5]:
processing_parameters = {
    "nasa_frp": "s3://s3vtaustralia/nasa_hotspots_gdf.geojson",
    "esa_frp": "s3://s3vtaustralia/s3vt_hotspots.geojson",
    "eumetsat_frp": "s3://s3vtaustralia/s3vt_eumetsat_hotspots.geojson",
    "landgate_frp": "s3://s3vtaustralia/landgate_hotspots_gdf.geojson",
    "sentinel3_swath_geojson": "s3://s3vtaustralia/sentinel3_swath_gdfs.geojson",
    "dea_frp": None,
    "lon_west": 113.0, # 147.0,
    "lat_south": -10.0, # -38.0,
    "lon_east": 154.0,
    "lat_north": -44.0, # -27.,
    "start_date": "2019-11-01",
    "end_date": "2019-11-30",
    "start_time": "20:00",
    "end_time": "03:00",
    "chunks": 250,
    "compare_field": "solar_night",
    "swath_config_file": Path("/home/jovyan/s3vtconfig.yaml"),
    "outdir": outdir,
    "test": False
}

In [ ]:
# This is to generate nearest .csv files. If .csv files already exists then skip this process. Takes around ~5-6 hours in this sandbox environment with 2-core and 16 GB RAM
nearest_hotspots_product_files = nearest_process.process_nearest_points(**processing_parameters)

## Nearest Hotspots DataFrame merged from neareast hotspots csv files

In [6]:
# csv directory is where nearest hotspots csv files are stored. 
csv_directory = outdir
print(csv_directory)
# This is read all the .csv files if name starts with `nearest_points` and ends with `compare_field` value from processing parameters.
nearest_hotspots_csv_files = [
    fp for fp in csv_directory.iterdir()
    if (fp.name.startswith("nearest_points"))
    and (fp.name.endswith("csv"))
]

/home/jovyan/s3vt_dask/s3vtdata/workdir_test1


In [7]:
# nearest points csv files that will be used analysis from here on.
nearest_hotspots_csv_files

[PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir_test1/nearest_points_TERRA_MODIS_LANDGATE_2000_0300.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir_test1/nearest_points_SUOMI NPP_VIIRS_NASA1_2000_0300.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir_test1/nearest_points_SENTINEL_3B_SLSTR_ESA_2000_0300.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir_test1/nearest_points_TERRA_MODIS_NASA6.03_2000_0300.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir_test1/nearest_points_NOAA 20_VIIRS_LANDGATE_2000_0300.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir_test1/nearest_points_SUOMI NPP_VIIRS_LANDGATE_2000_0300.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir_test1/nearest_points_AQUA_MODIS_LANDGATE_2000_0300.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir_test1/nearest_points_AQUA_MODIS_NASA6.03_2000_0300.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir_test1/nearest_points_SENTINEL_3A_SLSTR_ESA_2000_0300.c

In [8]:
# returns a dask DataFrame with index set at column `compare_field` from processing_parameters.
nearest_points_ddf = util.csv_to_dataframe(nearest_hotspots_csv_files, processing_parameters["compare_field"])

In [9]:
nearest_points_ddf = nearest_points_ddf.compute()

In [10]:
nearest_points_ddf

,Unnamed: 0,latitude,longitude,satellite,sensor,confidence,power,datetime,solar_day,satellite_sensor_product,...,2_power,2_datetime,2_solar_day,2_satellite_sensor_product,2_geometry,2_solar_night,dist,dist_m,timedelta,count
solar_night,,,,,,,,,,,,,,,,,,,,,
2019-10-31 12:09:53,906995,-31.721,117.224,NOAA 20,VIIRS,-1.0,-1.0,2019-10-31 16:21:00,2019-11-01 00:09:53,NOAA 20_VIIRS_LANDGATE,...,-1.0,2019-10-31 17:30:00,2019-11-01 01:26:10,AQUA_MODIS_LANDGATE,POINT (119.045 -23.385),2019-10-31 13:26:10,8.532187,941030.670662,0 days 01:09:00,1
2019-10-31 12:09:53,527601,-31.721,117.224,NOAA 20,VIIRS,-1.0,-1.0,2019-10-31 16:21:00,2019-11-01 00:09:53,NOAA 20_VIIRS_LANDGATE,...,-1.0,2019-10-31 16:24:00,2019-11-01 00:12:53,NOAA 20_VIIRS_LANDGATE,POINT (117.22438 -31.72068),2019-10-31 12:12:53,0.000000,0.000000,0 days 00:03:00,1
2019-10-31 12:09:53,675320,-31.721,117.224,NOAA 20,VIIRS,-1.0,-1.0,2019-10-31 16:21:00,2019-11-01 00:09:53,NOAA 20_VIIRS_LANDGATE,...,-1.0,2019-10-31 17:15:00,2019-11-01 01:03:54,SUOMI NPP_VIIRS_LANDGATE,POINT (117.22808 -31.72068),2019-10-31 13:03:54,0.003700,379.112619,0 days 00:54:00,1
2019-10-31 12:12:53,675321,-31.721,117.224,NOAA 20,VIIRS,-1.0,-1.0,2019-10-31 16:24:00,2019-11-01 00:12:53,NOAA 20_VIIRS_LANDGATE,...,-1.0,2019-10-31 17:15:00,2019-11-01 01:03:54,SUOMI NPP_VIIRS_LANDGATE,POINT (117.22808 -31.72068),2019-10-31 13:03:54,0.003700,379.112619,0 days 00:51:00,1
2019-10-31 12:12:53,527602,-31.721,117.224,NOAA 20,VIIRS,-1.0,-1.0,2019-10-31 16:24:00,2019-11-01 00:12:53,NOAA 20_VIIRS_LANDGATE,...,-1.0,2019-10-31 16:24:00,2019-11-01 00:12:53,NOAA 20_VIIRS_LANDGATE,POINT (117.22438 -31.72068),2019-10-31 12:12:53,0.000000,0.000000,0 days 00:00:00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-11-30 11:17:58,338985,-26.565,152.245,TERRA,MODIS,-1.0,-1.0,2019-11-30 13:09:00,2019-11-30 23:17:58,TERRA_MODIS_LANDGATE,...,-1.0,2019-11-30 13:09:00,2019-11-30 23:17:58,TERRA_MODIS_LANDGATE,POINT (152.245 -26.565),2019-11-30 11:17:58,0.000000,0.000000,0 days 00:00:00,1
2019-11-30 11:18:01,338986,-26.555,152.255,TERRA,MODIS,-1.0,-1.0,2019-11-30 13:09:00,2019-11-30 23:18:01,TERRA_MODIS_LANDGATE,...,-1.0,2019-11-30 13:09:00,2019-11-30 23:18:01,TERRA_MODIS_LANDGATE,POINT (152.255 -26.555),2019-11-30 11:18:01,0.000000,0.000000,0 days 00:00:00,1
2019-11-30 11:18:01,84758,-26.555,152.255,TERRA,MODIS,-1.0,-1.0,2019-11-30 13:09:00,2019-11-30 23:18:01,TERRA_MODIS_LANDGATE,...,10.2,2019-11-30 13:01:00,2019-11-30 23:10:00,TERRA_MODIS_NASA6.03,POINT (152.2518 -26.5467),2019-11-30 11:10:00,0.008896,973.324653,0 days 00:08:00,1


# Results
## Co-occurrence metrics

In [11]:
region_alias = "nsw"
output_directory = processing_parameters["outdir"]
comparison_prefix = (
    f"{processing_parameters['start_date'].replace('-', '')}"
    f"_{processing_parameters['end_date'].replace('-', '')}"
    f"_{processing_parameters['start_time'].replace(':','')}"
    f"_{processing_parameters['end_time'].replace(':','')}"
    f"_{region_alias}"
)

In [12]:
# set the nearest distance threshold between two hotspots to confine the analysis within the distance threshold.  
dist_threshold = 5000  # units in meters

In [13]:
nearest_ddf_dist_subset = nearest_points_ddf[nearest_points_ddf["dist_m"] < dist_threshold]

In [14]:
nearest_ddf_dist_subset

,Unnamed: 0,latitude,longitude,satellite,sensor,confidence,power,datetime,solar_day,satellite_sensor_product,...,2_power,2_datetime,2_solar_day,2_satellite_sensor_product,2_geometry,2_solar_night,dist,dist_m,timedelta,count
solar_night,,,,,,,,,,,,,,,,,,,,,
2019-10-31 12:09:53,527601,-31.721,117.224,NOAA 20,VIIRS,-1.0,-1.0,2019-10-31 16:21:00,2019-11-01 00:09:53,NOAA 20_VIIRS_LANDGATE,...,-1.0,2019-10-31 16:24:00,2019-11-01 00:12:53,NOAA 20_VIIRS_LANDGATE,POINT (117.22438 -31.72068),2019-10-31 12:12:53,0.000000,0.000000,0 days 00:03:00,1
2019-10-31 12:09:53,675320,-31.721,117.224,NOAA 20,VIIRS,-1.0,-1.0,2019-10-31 16:21:00,2019-11-01 00:09:53,NOAA 20_VIIRS_LANDGATE,...,-1.0,2019-10-31 17:15:00,2019-11-01 01:03:54,SUOMI NPP_VIIRS_LANDGATE,POINT (117.22808 -31.72068),2019-10-31 13:03:54,0.003700,379.112619,0 days 00:54:00,1
2019-10-31 12:12:53,675321,-31.721,117.224,NOAA 20,VIIRS,-1.0,-1.0,2019-10-31 16:24:00,2019-11-01 00:12:53,NOAA 20_VIIRS_LANDGATE,...,-1.0,2019-10-31 17:15:00,2019-11-01 01:03:54,SUOMI NPP_VIIRS_LANDGATE,POINT (117.22808 -31.72068),2019-10-31 13:03:54,0.003700,379.112619,0 days 00:51:00,1
2019-10-31 12:12:53,527602,-31.721,117.224,NOAA 20,VIIRS,-1.0,-1.0,2019-10-31 16:24:00,2019-11-01 00:12:53,NOAA 20_VIIRS_LANDGATE,...,-1.0,2019-10-31 16:24:00,2019-11-01 00:12:53,NOAA 20_VIIRS_LANDGATE,POINT (117.22438 -31.72068),2019-10-31 12:12:53,0.000000,0.000000,0 days 00:00:00,1
2019-10-31 12:13:53,675322,-31.721,117.224,NOAA 20,VIIRS,-1.0,-1.0,2019-10-31 16:25:00,2019-11-01 00:13:53,NOAA 20_VIIRS_LANDGATE,...,-1.0,2019-10-31 17:15:00,2019-11-01 01:03:54,SUOMI NPP_VIIRS_LANDGATE,POINT (117.22808 -31.72068),2019-10-31 13:03:54,0.003700,379.112619,0 days 00:50:00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-11-30 11:17:58,338985,-26.565,152.245,TERRA,MODIS,-1.0,-1.0,2019-11-30 13:09:00,2019-11-30 23:17:58,TERRA_MODIS_LANDGATE,...,-1.0,2019-11-30 13:09:00,2019-11-30 23:17:58,TERRA_MODIS_LANDGATE,POINT (152.245 -26.565),2019-11-30 11:17:58,0.000000,0.000000,0 days 00:00:00,1
2019-11-30 11:18:01,338986,-26.555,152.255,TERRA,MODIS,-1.0,-1.0,2019-11-30 13:09:00,2019-11-30 23:18:01,TERRA_MODIS_LANDGATE,...,-1.0,2019-11-30 13:09:00,2019-11-30 23:18:01,TERRA_MODIS_LANDGATE,POINT (152.255 -26.555),2019-11-30 11:18:01,0.000000,0.000000,0 days 00:00:00,1
2019-11-30 11:18:01,84758,-26.555,152.255,TERRA,MODIS,-1.0,-1.0,2019-11-30 13:09:00,2019-11-30 23:18:01,TERRA_MODIS_LANDGATE,...,10.2,2019-11-30 13:01:00,2019-11-30 23:10:00,TERRA_MODIS_NASA6.03,POINT (152.2518 -26.5467),2019-11-30 11:10:00,0.008896,973.324653,0 days 00:08:00,1


In [15]:
numerator = util.pandas_pivot_table(
    nearest_ddf_dist_subset,
    index=["satellite_sensor_product"],
    columns=["2_satellite_sensor_product"],
    values=["count"],
    aggfunc={"count": np.sum}
    
)

In [16]:
numerator.to_csv(output_directory.joinpath(f"{comparison_prefix}_matches_{dist_threshold}.csv"))
numerator

count                      \
2_satellite_sensor_product AQUA_MODIS_LANDGATE AQUA_MODIS_NASA6.03   
satellite_sensor_product                                             
AQUA_MODIS_LANDGATE                      48027               41568   
AQUA_MODIS_NASA6.03                       7596                7675   
NOAA 20_VIIRS_LANDGATE                  114671               95107   
NOAA-19_AVHRR_LANDGATE                   12039               10032   
SENTINEL_3A_SLSTR_ESA                     8691                6460   
SENTINEL_3B_SLSTR_ESA                    11851               10309   
SUOMI NPP_VIIRS_LANDGATE                146570              125488   
SUOMI NPP_VIIRS_NASA1                   105496               85983   
TERRA_MODIS_LANDGATE                     27977               20964   
TERRA_MODIS_NASA6.03                      6951                5562   

                                                                          \
2_satellite_sensor_product NOAA 20_VIIRS_LANDGATE NOAA-19_AVHRR_LANDGATE   
satellite_sensor_product                                                   
AQUA_MODIS_LANDGATE                         39763                  14548   
AQUA_MODIS_NASA6.03                          6449                   2467   
NOAA 20_VIIRS_LANDGATE                     147719                  56742   
NOAA-19_AVHRR_LANDGATE                      14201                  16905   
SENTINEL_3A_SLSTR_ESA                       11474                   6656   
SENTINEL_3B_SLSTR_ESA                       14503                   5114   
SUOMI NPP_VIIRS_LANDGATE                   143979                  43715   
SUOMI NPP_VIIRS_NASA1                      114758                  45870   
TERRA_MODIS_LANDGATE                        35278                  14211   
TERRA_MODIS_NASA6.03                         8473                   3851   

                                                                        \
2_satellite_sensor_product SENTINEL_3A_SLSTR_ESA SENTINEL_3B_SLSTR_ESA   
satellite_sensor_product                                                 
AQUA_MODIS_LANDGATE                        11392                 17629   
AQUA_MODIS_NASA6.03                         1643                  3498   
NOAA 20_VIIRS_LANDGATE                     46959                 47132   
NOAA-19_AVHRR_LANDGATE                      6706                  3941   
SENTINEL_3A_SLSTR_ESA                      14832                   275   
SENTINEL_3B_SLSTR_ESA                        481                 17223   
SUOMI NPP_VIIRS_LANDGATE                   31038                 67046   
SUOMI NPP_VIIRS_NASA1                      34225                 43806   
TERRA_MODIS_LANDGATE                       14130                  9523   
TERRA_MODIS_NASA6.03                        3066                  2558   

                                                                           \
2_satellite_sensor_product SUOMI NPP_VIIRS_LANDGATE SUOMI NPP_VIIRS_NASA1   
satellite_sensor_product                                                    
AQUA_MODIS_LANDGATE                           41359                 47258   
AQUA_MODIS_NASA6.03                            6977                  7637   
NOAA 20_VIIRS_LANDGATE                       116477                144431   
NOAA-19_AVHRR_LANDGATE                        12528                 15971   
SENTINEL_3A_SLSTR_ESA                          9164                 13961   
SENTINEL_3B_SLSTR_ESA                         14017                 16326   
SUOMI NPP_VIIRS_LANDGATE                     173583                173142   
SUOMI NPP_VIIRS_NASA1                        115377                136108   
TERRA_MODIS_LANDGATE                          30704                 40060   
TERRA_MODIS_NASA6.03                           7318                  9580   

                                                                      
2_satellite_sensor_product TERRA_MODIS_LANDGATE TERRA_MODIS_NASA6.03  
satellite_sensor_product          

In [17]:
denominator = util.pandas_pivot_table(
    nearest_points_ddf,
    index=["satellite_sensor_product"],
    columns=["2_satellite_sensor_product"],
    values=["count"],
    aggfunc={"count": np.sum}
    
)

In [18]:
denominator.to_csv(output_directory.joinpath(f"{comparison_prefix}_matches_count.csv"))
denominator

count                      \
2_satellite_sensor_product AQUA_MODIS_LANDGATE AQUA_MODIS_NASA6.03   
satellite_sensor_product                                             
AQUA_MODIS_LANDGATE                      48027               47953   
AQUA_MODIS_NASA6.03                       7675                7675   
NOAA 20_VIIRS_LANDGATE                  137023              136283   
NOAA-19_AVHRR_LANDGATE                   15096               15096   
SENTINEL_3A_SLSTR_ESA                    12810               12810   
SENTINEL_3B_SLSTR_ESA                    15874               15874   
SUOMI NPP_VIIRS_LANDGATE                169999              169723   
SUOMI NPP_VIIRS_NASA1                   128037              128037   
TERRA_MODIS_LANDGATE                     37995               37995   
TERRA_MODIS_NASA6.03                      8887                8887   

                                                                          \
2_satellite_sensor_product NOAA 20_VIIRS_LANDGATE NOAA-19_AVHRR_LANDGATE   
satellite_sensor_product                                                   
AQUA_MODIS_LANDGATE                         48027                  28975   
AQUA_MODIS_NASA6.03                          7675                   4359   
NOAA 20_VIIRS_LANDGATE                     147719                  83956   
NOAA-19_AVHRR_LANDGATE                      16759                  16905   
SENTINEL_3A_SLSTR_ESA                       14832                  10991   
SENTINEL_3B_SLSTR_ESA                       17223                   8289   
SUOMI NPP_VIIRS_LANDGATE                   173583                  93756   
SUOMI NPP_VIIRS_NASA1                      136108                  89730   
TERRA_MODIS_LANDGATE                        41915                  26537   
TERRA_MODIS_NASA6.03                         9825                   6597   

                                                                        \
2_satellite_sensor_product SENTINEL_3A_SLSTR_ESA SENTINEL_3B_SLSTR_ESA   
satellite_sensor_product                                                 
AQUA_MODIS_LANDGATE                        12433                 18539   
AQUA_MODIS_NASA6.03                         1795                  3625   
NOAA 20_VIIRS_LANDGATE                     50555                 50408   
NOAA-19_AVHRR_LANDGATE                      7043                  4254   
SENTINEL_3A_SLSTR_ESA                      14832                   356   
SENTINEL_3B_SLSTR_ESA                        591                 17223   
SUOMI NPP_VIIRS_LANDGATE                   35045                 70232   
SUOMI NPP_VIIRS_NASA1                      38459                 46456   
TERRA_MODIS_LANDGATE                       15096                  9930   
TERRA_MODIS_NASA6.03                        3276                  2619   

                                                                           \
2_satellite_sensor_product SUOMI NPP_VIIRS_LANDGATE SUOMI NPP_VIIRS_NASA1   
satellite_sensor_product                                                    
AQUA_MODIS_LANDGATE                           48027                 47953   
AQUA_MODIS_NASA6.03                            7675                  7675   
NOAA 20_VIIRS_LANDGATE                       147719                146961   
NOAA-19_AVHRR_LANDGATE                        15252                 16791   
SENTINEL_3A_SLSTR_ESA                         14832                 14832   
SENTINEL_3B_SLSTR_ESA                         17186                 17186   
SUOMI NPP_VIIRS_LANDGATE                     173583                173300   
SUOMI NPP_VIIRS_NASA1                        136108                136108   
TERRA_MODIS_LANDGATE                          41915                 41915   
TERRA_MODIS_NASA6.03                           9825                  9825   

                                                                      
2_satellite_sensor_product TERRA_MODIS_LANDGATE TERRA_MODIS_NASA6.03  
satellite_sensor_product          

In [19]:
# Difference of matched points closer than 5000m
difference = denominator - numerator

In [20]:
difference.to_csv(output_directory.joinpath(f"{comparison_prefix}_count_difference.csv"))
difference

count                      \
2_satellite_sensor_product AQUA_MODIS_LANDGATE AQUA_MODIS_NASA6.03   
satellite_sensor_product                                             
AQUA_MODIS_LANDGATE                          0                6385   
AQUA_MODIS_NASA6.03                         79                   0   
NOAA 20_VIIRS_LANDGATE                   22352               41176   
NOAA-19_AVHRR_LANDGATE                    3057                5064   
SENTINEL_3A_SLSTR_ESA                     4119                6350   
SENTINEL_3B_SLSTR_ESA                     4023                5565   
SUOMI NPP_VIIRS_LANDGATE                 23429               44235   
SUOMI NPP_VIIRS_NASA1                    22541               42054   
TERRA_MODIS_LANDGATE                     10018               17031   
TERRA_MODIS_NASA6.03                      1936                3325   

                                                                          \
2_satellite_sensor_product NOAA 20_VIIRS_LANDGATE NOAA-19_AVHRR_LANDGATE   
satellite_sensor_product                                                   
AQUA_MODIS_LANDGATE                          8264                  14427   
AQUA_MODIS_NASA6.03                          1226                   1892   
NOAA 20_VIIRS_LANDGATE                          0                  27214   
NOAA-19_AVHRR_LANDGATE                       2558                      0   
SENTINEL_3A_SLSTR_ESA                        3358                   4335   
SENTINEL_3B_SLSTR_ESA                        2720                   3175   
SUOMI NPP_VIIRS_LANDGATE                    29604                  50041   
SUOMI NPP_VIIRS_NASA1                       21350                  43860   
TERRA_MODIS_LANDGATE                         6637                  12326   
TERRA_MODIS_NASA6.03                         1352                   2746   

                                                                        \
2_satellite_sensor_product SENTINEL_3A_SLSTR_ESA SENTINEL_3B_SLSTR_ESA   
satellite_sensor_product                                                 
AQUA_MODIS_LANDGATE                         1041                   910   
AQUA_MODIS_NASA6.03                          152                   127   
NOAA 20_VIIRS_LANDGATE                      3596                  3276   
NOAA-19_AVHRR_LANDGATE                       337                   313   
SENTINEL_3A_SLSTR_ESA                          0                    81   
SENTINEL_3B_SLSTR_ESA                        110                     0   
SUOMI NPP_VIIRS_LANDGATE                    4007                  3186   
SUOMI NPP_VIIRS_NASA1                       4234                  2650   
TERRA_MODIS_LANDGATE                         966                   407   
TERRA_MODIS_NASA6.03                         210                    61   

                                                                           \
2_satellite_sensor_product SUOMI NPP_VIIRS_LANDGATE SUOMI NPP_VIIRS_NASA1   
satellite_sensor_product                                                    
AQUA_MODIS_LANDGATE                            6668                   695   
AQUA_MODIS_NASA6.03                             698                    38   
NOAA 20_VIIRS_LANDGATE                        31242                  2530   
NOAA-19_AVHRR_LANDGATE                         2724                   820   
SENTINEL_3A_SLSTR_ESA                          5668                   871   
SENTINEL_3B_SLSTR_ESA                          3169                   860   
SUOMI NPP_VIIRS_LANDGATE                          0                   158   
SUOMI NPP_VIIRS_NASA1                         20731                     0   
TERRA_MODIS_LANDGATE                          11211                  1855   
TERRA_MODIS_NASA6.03                           2507                   245   

                                                                      
2_satellite_sensor_product TERRA_MODIS_LANDGATE TERRA_MODIS_NASA6.03  
satellite_sensor_product          

In [21]:
# Percentage of matched points closer than dist_threshold
percentage = (numerator / denominator) * 100
percentage = np.round(percentage, 2)

In [22]:
percentage.to_csv(output_directory.joinpath(f"{comparison_prefix}_percentage.csv"))
percentage

count                      \
2_satellite_sensor_product AQUA_MODIS_LANDGATE AQUA_MODIS_NASA6.03   
satellite_sensor_product                                             
AQUA_MODIS_LANDGATE                     100.00               86.68   
AQUA_MODIS_NASA6.03                      98.97              100.00   
NOAA 20_VIIRS_LANDGATE                   83.69               69.79   
NOAA-19_AVHRR_LANDGATE                   79.75               66.45   
SENTINEL_3A_SLSTR_ESA                    67.85               50.43   
SENTINEL_3B_SLSTR_ESA                    74.66               64.94   
SUOMI NPP_VIIRS_LANDGATE                 86.22               73.94   
SUOMI NPP_VIIRS_NASA1                    82.39               67.15   
TERRA_MODIS_LANDGATE                     73.63               55.18   
TERRA_MODIS_NASA6.03                     78.22               62.59   

                                                                          \
2_satellite_sensor_product NOAA 20_VIIRS_LANDGATE NOAA-19_AVHRR_LANDGATE   
satellite_sensor_product                                                   
AQUA_MODIS_LANDGATE                         82.79                  50.21   
AQUA_MODIS_NASA6.03                         84.03                  56.60   
NOAA 20_VIIRS_LANDGATE                     100.00                  67.59   
NOAA-19_AVHRR_LANDGATE                      84.74                 100.00   
SENTINEL_3A_SLSTR_ESA                       77.36                  60.56   
SENTINEL_3B_SLSTR_ESA                       84.21                  61.70   
SUOMI NPP_VIIRS_LANDGATE                    82.95                  46.63   
SUOMI NPP_VIIRS_NASA1                       84.31                  51.12   
TERRA_MODIS_LANDGATE                        84.17                  53.55   
TERRA_MODIS_NASA6.03                        86.24                  58.38   

                                                                        \
2_satellite_sensor_product SENTINEL_3A_SLSTR_ESA SENTINEL_3B_SLSTR_ESA   
satellite_sensor_product                                                 
AQUA_MODIS_LANDGATE                        91.63                 95.09   
AQUA_MODIS_NASA6.03                        91.53                 96.50   
NOAA 20_VIIRS_LANDGATE                     92.89                 93.50   
NOAA-19_AVHRR_LANDGATE                     95.22                 92.64   
SENTINEL_3A_SLSTR_ESA                     100.00                 77.25   
SENTINEL_3B_SLSTR_ESA                      81.39                100.00   
SUOMI NPP_VIIRS_LANDGATE                   88.57                 95.46   
SUOMI NPP_VIIRS_NASA1                      88.99                 94.30   
TERRA_MODIS_LANDGATE                       93.60                 95.90   
TERRA_MODIS_NASA6.03                       93.59                 97.67   

                                                                           \
2_satellite_sensor_product SUOMI NPP_VIIRS_LANDGATE SUOMI NPP_VIIRS_NASA1   
satellite_sensor_product                                                    
AQUA_MODIS_LANDGATE                           86.12                 98.55   
AQUA_MODIS_NASA6.03                           90.91                 99.50   
NOAA 20_VIIRS_LANDGATE                        78.85                 98.28   
NOAA-19_AVHRR_LANDGATE                        82.14                 95.12   
SENTINEL_3A_SLSTR_ESA                         61.79                 94.13   
SENTINEL_3B_SLSTR_ESA                         81.56                 95.00   
SUOMI NPP_VIIRS_LANDGATE                     100.00                 99.91   
SUOMI NPP_VIIRS_NASA1                         84.77                100.00   
TERRA_MODIS_LANDGATE                          73.25                 95.57   
TERRA_MODIS_NASA6.03                          74.48                 97.51   

                                                                      
2_satellite_sensor_product TERRA_MODIS_LANDGATE TERRA_MODIS_NASA6.03  
satellite_sensor_product          

In [23]:
# Maximum time between match points < dist_threshold
timemax = util.pandas_pivot_table(
    nearest_ddf_dist_subset,
    index=["satellite_sensor_product"],
    columns=["2_satellite_sensor_product"],
    values=["timedelta"],
    aggfunc={"timedelta": np.max}
    
)

In [24]:
timemax.to_csv(output_directory.joinpath(f"{comparison_prefix}_max_time_matched_points.csv"))
timemax

timedelta                      \
2_satellite_sensor_product AQUA_MODIS_LANDGATE AQUA_MODIS_NASA6.03   
satellite_sensor_product                                             
AQUA_MODIS_LANDGATE            0 days 00:16:00     0 days 01:44:00   
AQUA_MODIS_NASA6.03            0 days 01:44:00     0 days 00:00:00   
NOAA 20_VIIRS_LANDGATE         0 days 01:58:00     0 days 01:53:00   
NOAA-19_AVHRR_LANDGATE         0 days 06:42:00     0 days 04:20:00   
SENTINEL_3A_SLSTR_ESA          0 days 03:59:53     0 days 04:00:12   
SENTINEL_3B_SLSTR_ESA          0 days 04:11:31     0 days 03:51:50   
SUOMI NPP_VIIRS_LANDGATE       0 days 02:03:00     0 days 01:43:00   
SUOMI NPP_VIIRS_NASA1          0 days 01:57:00     0 days 01:51:00   
TERRA_MODIS_LANDGATE           0 days 03:28:00     0 days 03:22:00   
TERRA_MODIS_NASA6.03           0 days 03:21:00     0 days 03:16:00   

                                                                          \
2_satellite_sensor_product NOAA 20_VIIRS_LANDGATE NOAA-19_AVHRR_LANDGATE   
satellite_sensor_product                                                   
AQUA_MODIS_LANDGATE               0 days 01:58:00        0 days 06:56:00   
AQUA_MODIS_NASA6.03               0 days 01:44:00        0 days 04:20:00   
NOAA 20_VIIRS_LANDGATE            0 days 01:46:00        0 days 06:49:00   
NOAA-19_AVHRR_LANDGATE            0 days 06:46:00        0 days 00:07:00   
SENTINEL_3A_SLSTR_ESA             0 days 04:12:18        0 days 06:55:39   
SENTINEL_3B_SLSTR_ESA             0 days 04:07:59        0 days 07:13:56   
SUOMI NPP_VIIRS_LANDGATE          0 days 00:59:00        0 days 07:36:00   
SUOMI NPP_VIIRS_NASA1             0 days 01:02:00        0 days 07:45:00   
TERRA_MODIS_LANDGATE              0 days 04:08:00        0 days 07:03:00   
TERRA_MODIS_NASA6.03              0 days 04:00:00        0 days 06:54:00   

                                                                        \
2_satellite_sensor_product SENTINEL_3A_SLSTR_ESA SENTINEL_3B_SLSTR_ESA   
satellite_sensor_product                                                 
AQUA_MODIS_LANDGATE              0 days 03:59:53       0 days 04:11:30   
AQUA_MODIS_NASA6.03              0 days 04:00:12       0 days 03:51:50   
NOAA 20_VIIRS_LANDGATE           0 days 04:12:18       0 days 04:10:58   
NOAA-19_AVHRR_LANDGATE           0 days 07:00:39       0 days 07:13:56   
SENTINEL_3A_SLSTR_ESA            0 days 00:00:01       0 days 00:39:54   
SENTINEL_3B_SLSTR_ESA            0 days 00:39:55       0 days 00:00:01   
SUOMI NPP_VIIRS_LANDGATE         0 days 04:08:30       0 days 04:14:07   
SUOMI NPP_VIIRS_NASA1            0 days 04:11:30       0 days 04:15:07   
TERRA_MODIS_LANDGATE             0 days 01:39:34       0 days 01:28:31   
TERRA_MODIS_NASA6.03             0 days 01:29:51       0 days 01:21:17   

                                                                           \
2_satellite_sensor_product SUOMI NPP_VIIRS_LANDGATE SUOMI NPP_VIIRS_NASA1   
satellite_sensor_product                                                    
AQUA_MODIS_LANDGATE                 0 days 01:49:00       0 days 01:57:00   
AQUA_MODIS_NASA6.03                 0 days 01:32:00       0 days 01:51:00   
NOAA 20_VIIRS_LANDGATE              0 days 00:59:00       0 days 01:02:00   
NOAA-19_AVHRR_LANDGATE              0 days 07:36:00       0 days 07:45:00   
SENTINEL_3A_SLSTR_ESA               0 days 04:08:30       0 days 04:11:30   
SENTINEL_3B_SLSTR_ESA               0 days 04:11:07       0 days 04:15:07   
SUOMI NPP_VIIRS_LANDGATE            0 days 01:47:00       0 days 01:51:00   
SUOMI NPP_VIIRS_NASA1               0 days 01:51:00       0 days 00:00:00   
TERRA_MODIS_LANDGATE                0 days 04:18:00       0 days 04:21:00   
TERRA_MODIS_NASA6.03                0 days 04:08:00       0 days 04:10:00   

                                                                      
2_satellite_sensor_product TERRA_MODIS_LANDGATE TERRA_MODIS_NASA6.03  
satellite_sensor_product      

In [25]:
# Minimum time between match points < dist_threshold
timemin = util.pandas_pivot_table(
    nearest_ddf_dist_subset,
    index=["satellite_sensor_product"],
    columns=["2_satellite_sensor_product"],
    values=["timedelta"],
    aggfunc={"timedelta": np.min}
)

In [26]:
timemin.to_csv(output_directory.joinpath(f"{comparison_prefix}_min_time_matched_points.csv"))
timemin

timedelta                      \
2_satellite_sensor_product AQUA_MODIS_LANDGATE AQUA_MODIS_NASA6.03   
satellite_sensor_product                                             
AQUA_MODIS_LANDGATE            0 days 00:00:00     0 days 00:00:00   
AQUA_MODIS_NASA6.03            0 days 00:00:00     0 days 00:00:00   
NOAA 20_VIIRS_LANDGATE         0 days 00:00:00     0 days 00:00:00   
NOAA-19_AVHRR_LANDGATE         0 days 01:47:00     0 days 01:49:00   
SENTINEL_3A_SLSTR_ESA          0 days 01:43:46     0 days 01:53:00   
SENTINEL_3B_SLSTR_ESA          0 days 02:00:31     0 days 02:06:31   
SUOMI NPP_VIIRS_LANDGATE       0 days 00:00:00     0 days 00:00:00   
SUOMI NPP_VIIRS_NASA1          0 days 00:00:00     0 days 00:00:00   
TERRA_MODIS_LANDGATE           0 days 01:16:00     0 days 01:21:00   
TERRA_MODIS_NASA6.03           0 days 01:21:00     0 days 01:26:00   

                                                                          \
2_satellite_sensor_product NOAA 20_VIIRS_LANDGATE NOAA-19_AVHRR_LANDGATE   
satellite_sensor_product                                                   
AQUA_MODIS_LANDGATE               0 days 00:00:00        0 days 01:43:00   
AQUA_MODIS_NASA6.03               0 days 00:00:00        0 days 01:52:00   
NOAA 20_VIIRS_LANDGATE            0 days 00:00:00        0 days 01:50:00   
NOAA-19_AVHRR_LANDGATE            0 days 01:50:00        0 days 00:00:00   
SENTINEL_3A_SLSTR_ESA             0 days 00:46:58        0 days 05:14:19   
SENTINEL_3B_SLSTR_ESA             0 days 01:20:06        0 days 05:22:30   
SUOMI NPP_VIIRS_LANDGATE          0 days 00:40:00        0 days 01:58:00   
SUOMI NPP_VIIRS_NASA1             0 days 00:38:00        0 days 01:51:00   
TERRA_MODIS_LANDGATE              0 days 00:22:00        0 days 04:13:00   
TERRA_MODIS_NASA6.03              0 days 00:28:00        0 days 04:24:00   

                                                                        \
2_satellite_sensor_product SENTINEL_3A_SLSTR_ESA SENTINEL_3B_SLSTR_ESA   
satellite_sensor_product                                                 
AQUA_MODIS_LANDGATE              0 days 01:34:47       0 days 01:58:31   
AQUA_MODIS_NASA6.03              0 days 01:53:00       0 days 02:06:31   
NOAA 20_VIIRS_LANDGATE           0 days 00:46:58       0 days 01:20:06   
NOAA-19_AVHRR_LANDGATE           0 days 05:14:19       0 days 05:22:30   
SENTINEL_3A_SLSTR_ESA            0 days 00:00:00       0 days 00:38:56   
SENTINEL_3B_SLSTR_ESA            0 days 00:38:56       0 days 00:00:00   
SUOMI NPP_VIIRS_LANDGATE         0 days 01:26:10       0 days 01:27:32   
SUOMI NPP_VIIRS_NASA1            0 days 01:33:10       0 days 01:33:31   
TERRA_MODIS_LANDGATE             0 days 00:00:02       0 days 00:00:00   
TERRA_MODIS_NASA6.03             0 days 00:00:06       0 days 00:01:07   

                                                                           \
2_satellite_sensor_product SUOMI NPP_VIIRS_LANDGATE SUOMI NPP_VIIRS_NASA1   
satellite_sensor_product                                                    
AQUA_MODIS_LANDGATE                 0 days 00:00:00       0 days 00:00:00   
AQUA_MODIS_NASA6.03                 0 days 00:00:00       0 days 00:00:00   
NOAA 20_VIIRS_LANDGATE              0 days 00:40:00       0 days 00:38:00   
NOAA-19_AVHRR_LANDGATE              0 days 01:58:00       0 days 01:51:00   
SENTINEL_3A_SLSTR_ESA               0 days 01:27:10       0 days 01:33:10   
SENTINEL_3B_SLSTR_ESA               0 days 01:27:33       0 days 01:33:32   
SUOMI NPP_VIIRS_LANDGATE            0 days 00:00:00       0 days 00:00:00   
SUOMI NPP_VIIRS_NASA1               0 days 00:00:00       0 days 00:00:00   
TERRA_MODIS_LANDGATE                0 days 00:33:00       0 days 00:34:00   
TERRA_MODIS_NASA6.03                0 days 00:40:00       0 days 00:43:00   

                                                                      
2_satellite_sensor_product TERRA_MODIS_LANDGATE TERRA_MODIS_NASA6.03  
satellite_sensor_product      

In [27]:
# Average distance (m) between matched points < dist_threshold
averagedist = util.pandas_pivot_table(
    nearest_ddf_dist_subset,
    index=["satellite_sensor_product"],
    columns=["2_satellite_sensor_product"],
    values=["dist_m"],
    aggfunc={"dist_m": np.mean}
)

In [28]:
averagedist = np.round(averagedist, 2)
averagedist.to_csv(output_directory.joinpath(f"{comparison_prefix}_average_distance_{dist_threshold}m.csv"))
averagedist

dist_m                      \
2_satellite_sensor_product AQUA_MODIS_LANDGATE AQUA_MODIS_NASA6.03   
satellite_sensor_product                                             
AQUA_MODIS_LANDGATE                       0.00              897.63   
AQUA_MODIS_NASA6.03                     131.98                0.00   
NOAA 20_VIIRS_LANDGATE                  816.14             1168.05   
NOAA-19_AVHRR_LANDGATE                 1035.99             1650.37   
SENTINEL_3A_SLSTR_ESA                  1271.49             1705.30   
SENTINEL_3B_SLSTR_ESA                  1239.23             1583.98   
SUOMI NPP_VIIRS_LANDGATE                784.02             1145.95   
SUOMI NPP_VIIRS_NASA1                   910.02             1345.97   
TERRA_MODIS_LANDGATE                   1093.86             1591.71   
TERRA_MODIS_NASA6.03                   1009.44             1376.56   

                                                                          \
2_satellite_sensor_product NOAA 20_VIIRS_LANDGATE NOAA-19_AVHRR_LANDGATE   
satellite_sensor_product                                                   
AQUA_MODIS_LANDGATE                        669.43                1083.63   
AQUA_MODIS_NASA6.03                        420.55                1073.76   
NOAA 20_VIIRS_LANDGATE                       0.00                1092.69   
NOAA-19_AVHRR_LANDGATE                     917.73                   0.00   
SENTINEL_3A_SLSTR_ESA                      848.03                1617.10   
SENTINEL_3B_SLSTR_ESA                     1040.72                1461.21   
SUOMI NPP_VIIRS_LANDGATE                   420.33                1192.34   
SUOMI NPP_VIIRS_NASA1                      556.58                1231.46   
TERRA_MODIS_LANDGATE                       693.95                1178.88   
TERRA_MODIS_NASA6.03                       538.04                1121.49   

                                                                        \
2_satellite_sensor_product SENTINEL_3A_SLSTR_ESA SENTINEL_3B_SLSTR_ESA   
satellite_sensor_product                                                 
AQUA_MODIS_LANDGATE                       869.91                831.66   
AQUA_MODIS_NASA6.03                       673.88                657.85   
NOAA 20_VIIRS_LANDGATE                    597.31                667.93   
NOAA-19_AVHRR_LANDGATE                    964.42               1007.79   
SENTINEL_3A_SLSTR_ESA                       0.00                783.13   
SENTINEL_3B_SLSTR_ESA                    1260.75                  0.00   
SUOMI NPP_VIIRS_LANDGATE                  664.39                608.08   
SUOMI NPP_VIIRS_NASA1                     632.90                627.54   
TERRA_MODIS_LANDGATE                      689.97                773.23   
TERRA_MODIS_NASA6.03                      574.61                550.74   

                                                                           \
2_satellite_sensor_product SUOMI NPP_VIIRS_LANDGATE SUOMI NPP_VIIRS_NASA1   
satellite_sensor_product                                                    
AQUA_MODIS_LANDGATE                          682.59                548.76   
AQUA_MODIS_NASA6.03                          493.67                343.09   
NOAA 20_VIIRS_LANDGATE                       383.23                283.98   
NOAA-19_AVHRR_LANDGATE                      1033.99                859.91   
SENTINEL_3A_SLSTR_ESA                       1034.00                675.20   
SENTINEL_3B_SLSTR_ESA                       1045.15                824.74   
SUOMI NPP_VIIRS_LANDGATE                       0.00                197.23   
SUOMI NPP_VIIRS_NASA1                        414.09                  0.00   
TERRA_MODIS_LANDGATE                         722.22                553.40   
TERRA_MODIS_NASA6.03                         553.01                396.41   

                                                                      
2_satellite_sensor_product TERRA_MODIS_LANDGATE TERRA_MODIS_NASA6.03  
satellite_sensor_product         

In [29]:
client.close() # close dask.distributed client